In [1]:
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
rating = pd.read_csv("rating 2.csv")
anime = pd.read_csv('anime.csv')
df = pd.merge(rating,anime.drop("rating", axis=1), on='anime_id')
df.head()

,user_id,anime_id,rating,name,genre,type,episodes,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
2,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
3,6,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
4,10,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297


In [3]:
df.groupby('name')['rating'].mean().sort_values(ascending=False).head(10)

name
Choegang Top Plate                                                  10.000000
STAR BEAT!: Hoshi no Kodou                                          10.000000
Shiroi Zou                                                          10.000000
Warui no wo Taose!! Salaryman Man                                   10.000000
Yakushiji Ryouko no Kaiki Jikenbo: Hamachou, Voice &amp; Fiction     9.000000
Tang Lang Bu Chan                                                    9.000000
Shiranpuri                                                           9.000000
Doukyuusei                                                           9.000000
Steins;Gate 0                                                        8.500000
Kimi no Na wa.                                                       8.297863
Name: rating, dtype: float64

In [4]:
df.groupby('name')['rating'].count().sort_values(ascending=False).head(10)

name
Death Note                          39340
Sword Art Online                    30583
Shingeki no Kyojin                  29584
Code Geass: Hangyaku no Lelouch     27718
Elfen Lied                          27506
Angel Beats!                        27183
Naruto                              25925
Fullmetal Alchemist                 25032
Fullmetal Alchemist: Brotherhood    24574
Toradora!                           24283
Name: rating, dtype: int64

In [5]:
ratings = pd.DataFrame(df.groupby('name')['rating'].mean())
ratings['num of ratings'] = pd.DataFrame(df.groupby('name')['rating'].count())
print(ratings)

                                                      rating  num of ratings
name                                                                        
&quot;0&quot;                                       2.769231              26
&quot;Aesop&quot; no Ohanashi yori: Ushi to Kae...  0.000000               2
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hats...  5.774936             782
&quot;Bungaku Shoujo&quot; Memoire                  6.155748             809
&quot;Bungaku Shoujo&quot; Movie                    6.457980            1535
...                                                      ...             ...
xxxHOLiC Kei                                        6.720774            3413
xxxHOLiC Movie: Manatsu no Yoru no Yume             6.313742            2365
xxxHOLiC Rou                                        6.403173            1513
xxxHOLiC Shunmuki                                   6.238602            1974
◯                                                   3.000000               6

In [6]:
genre_dict = pd.DataFrame(data=anime[['name','genre']])
genre_dict.set_index('name',inplace=True)
print(genre_dict)

                                                                                                genre
name                                                                                                 
Kimi no Na wa.                                                   Drama, Romance, School, Supernatural
Fullmetal Alchemist: Brotherhood                    Action, Adventure, Drama, Fantasy, Magic, Mili...
Gintama°                                            Action, Comedy, Historical, Parody, Samurai, S...
Steins;Gate                                                                          Sci-Fi, Thriller
Gintama&#039;                                       Action, Comedy, Historical, Parody, Samurai, S...
...                                                                                               ...
Toushindai My Lover: Minami tai Mecha-Minami                                                   Hentai
Under World                                                                       

In [19]:
def check_genre(genre_list,string):
    if any(x in string for x in genre_list):
        return True
    else:
        return False

def get_recommendation(name):
    #generating list of anime with the same genre with target
    anime_genre = genre_dict.loc[name].values[0].split(', ')
    cols = anime[anime['genre'].apply(
        lambda x: check_genre(anime_genre,str(x)))]['name'].tolist()

    #create matrix based on generated list
    animemat = df[df['name'].isin(cols)].pivot_table(
        index='user_id',columns='name',values='rating')

    #create correlation table
    anime_user_rating = animemat[name]
    similiar_anime = animemat.corrwith(anime_user_rating)
    corr_anime = pd.DataFrame(similiar_anime,columns=['correlation'])
    corr_anime = corr_anime.join(ratings['num of ratings'])
    corr_anime.dropna(inplace=True)
    corr_anime = corr_anime[corr_anime['num of ratings']>5000].sort_values(
        'correlation',ascending=False)

    return corr_anime.head(10)

In [20]:
get_recommendation('Naruto')

/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/alibiserikbay/Pyc

,correlation,num of ratings
Naruto,1.000000,25925
Bleach,0.720531,12845
InuYasha,0.704638,9328
Shaman King,0.696407,7814
Katekyo Hitman Reborn!,0.694202,6739
Tenjou Tenge,0.691476,5150
Sekirei,0.691413,7924
Rurouni Kenshin: Meiji Kenkaku Romantan,0.690855,7900
Shijou Saikyou no Deshi Kenichi,0.690250,5654
Tsubasa Chronicle,0.688495,5465


In [21]:
get_recommendation("Death Note")

/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/alibiserikbay/Pyc

,correlation,num of ratings
Death Note,1.000000,39340
Claymore,0.713321,12254
Higurashi no Naku Koro ni Kai,0.708664,9167
Beelzebub,0.707432,5871
Darker than Black: Kuro no Keiyakusha,0.705980,15591
Blood+,0.705211,7496
Soul Eater,0.702571,19759
Ga-Rei: Zero,0.698991,5351
Higurashi no Naku Koro ni,0.695316,12901
Elfen Lied,0.692775,27506


In [22]:
get_recommendation("Bleach")

/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/alibiserikbay/PycharmProjects/newproject/venv/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/alibiserikbay/Pychar

,correlation,num of ratings
Bleach,1.000000,12845
Katekyo Hitman Reborn!,0.729857,6739
Ben-To,0.728137,5691
Sekirei: Pure Engagement,0.727271,5221
Beelzebub,0.721067,5871
Naruto,0.720531,25925
Ao no Exorcist,0.719117,21204
Fairy Tail,0.716337,12344
Campione!: Matsurowanu Kamigami to Kamigoroshi no Maou,0.712311,5786
Black★Rock Shooter (TV),0.704002,7594
